# 🖼️ Harvard’s Artifacts Collections: Extract transform load [ETL], SQL Analytics 

### API Integration & Data Extraction


In [ ]:
import streamlit as st
harvard_api = st.secrets.get("harvard")["api_key"]
if not harvard_api:
    st.error("API key for Harvard not found in secrets.")
    st.stop()
url='https://api.harvardartmuseums.org/classification'
import requests
parameters={'apikey':harvard_api,
           'size':100}
response= requests.get(url, parameters)
response

In [ ]:
data=response.json()
data

In [ ]:
from collections import Counter
classiresults= [record['name']
                for record in data['records']
                if record.get('objectcount',0) >= 2500]
print('name with object count >= 2500:')
for name in classiresults:
    print(name)

In [ ]:
url = 'https://api.harvardartmuseums.org/object'
all_records = []   
classifications = ["Paintings", "Sculpture", "Drawings", "Fragments", "Photographs"]
for classi in classifications:
    print(f"\nFetching classification: {classi}")
    for page in range(1, 26):
        parameters = {
            'apikey': harvard_api,
            'size': 100,
            'page': page,
            'classification': classi
        }
        response = requests.get(url, parameters)
        data = response.json()
        all_records.extend(data['records'])
    print(f"Total collected so far: {len(all_records)}")

In [ ]:
data['records']

In [ ]:
print(type(data['records']))

In [ ]:
data['records'][0]

In [ ]:
artifact_metadata = []
artifact_media = []
artifact_colors = []
for i in all_records:
    artifact_metadata.append(dict(
        id = i['id'],
        title = i['title'],
        culture = i['culture'],
        period = i['period'],
        century = i['century'],
        medium = i['medium'],
        dimensions = i['dimensions'],
        description = i['description'],
        department = i['department'],
        classification = i['classification'],
        accessionyear = i['accessionyear'],
        accessionmethod = i['accessionmethod']))
    artifact_media.append(dict(
        objectid = i['objectid'],
        imagecount = i['imagecount'],
        mediacount = i['mediacount'],
        colorcount = i['colorcount'],
        rank_value = i['rank'],
        datebegin = i['datebegin'],
        dateend = i['dateend']))
    color_details = i.get('colors')
    if color_details:
        for j in color_details:
            artifact_colors.append(dict(
                objectid = i['objectid'],
                color = j['color'],
                spectrum = j['spectrum'],
                hue = j['hue'],
                percent = j['percent'],
                css3 = j['css3']))


In [ ]:
len(artifact_metadata)

In [ ]:
len(artifact_media)

In [ ]:
len(artifact_colors)

In [ ]:
artifact_metadata

In [ ]:
artifact_media

In [ ]:
artifact_colors

In [ ]:
import mysql.connector

In [ ]:
connection = mysql.connector.connect(
    host=st.secrets["mysql"]["host"],
    user=st.secrets["mysql"]["user"],
    password=st.secrets["mysql"]["password"],
    database=st.secrets["mysql"]["database"]
)
cursor = connection.cursor()
print("MySQL server established")

In [ ]:
cursor.execute('CREATE DATABASE IF NOT EXISTS harvard_artifacts;')
cursor.execute("USE harvard_artifacts;")
print("Database ready: harvard_artifacts")

In [ ]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS artifact_metadata (
  id INT PRIMARY KEY,
  title TEXT,
  culture TEXT,
  period TEXT,
  century TEXT,
  medium TEXT,
  dimensions TEXT,
  description TEXT,
  department TEXT,
  classification TEXT,
  accessionyear INT,
  accessionmethod TEXT
);
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS artifact_media (
  objectid INT PRIMARY KEY,
  imagecount INT,
  mediacount INT,
  colorcount INT,
  rank_value INT,
  datebegin INT,
  dateend INT,
  CONSTRAINT fk1_id FOREIGN KEY (objectid) REFERENCES artifact_metadata(id)
);
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS artifact_colors (
  objectid INT,
  color TEXT,
  spectrum TEXT,
  hue TEXT,
  percent REAL,
  css3 TEXT,
  CONSTRAINT fk2_id FOREIGN KEY (objectid) REFERENCES artifact_metadata(id)
);
""")
connection.commit()
print("Tables created successfully")

In [ ]:
cursor.execute("DELETE FROM artifact_colors;")
cursor.execute("DELETE FROM artifact_media;")
cursor.execute("DELETE FROM artifact_metadata;")
connection.commit()
print("Tables emptied Ready for fresh insert.")

In [ ]:
insert_metadata = """ 
INSERT INTO artifact_metadata ( id, title, culture, period, century, medium, dimensions, description, 
department, classification, accessionyear, accessionmethod ) 
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s) 
"""
cursor.executemany(insert_metadata, [(
    r.get('id'), 
    r.get('title'), 
    r.get('culture'), 
    r.get('period'), 
    r.get('century'),
    r.get('medium'), 
    r.get('dimensions'), 
    r.get('description'), 
    r.get('department'),
    r.get('classification'), 
    r.get('accessionyear'), 
    r.get('accessionmethod')
) for r in artifact_metadata])
connection.commit()
print("artifact_metadata inserted")

In [ ]:
insert_media = """
INSERT INTO artifact_media (
    objectid, imagecount, mediacount, colorcount, rank_value, datebegin, dateend
) VALUES (%s, %s, %s, %s, %s, %s, %s)
"""
cursor.executemany(insert_media, [(
            r.get('objectid'),
            r.get('imagecount'),
            r.get('mediacount'),
            r.get('colorcount'),
            r.get('rank_value'),  
            r.get('datebegin'),
            r.get('dateend')
        )for r in artifact_media])
connection.commit()
print("artifact_media inserted")

In [ ]:
insert_colors = """
INSERT INTO artifact_colors (
    objectid, color, spectrum, hue, percent, css3
) VALUES (%s, %s, %s, %s, %s, %s)
"""
cursor.executemany(insert_colors,[(
            r.get('objectid'),
            r.get('color'),
            r.get('spectrum'),
            r.get('hue'),
            r.get('percent'),
            r.get('css3')
        )for r in artifact_colors])
connection.commit()
print("artifact_colors inserted")

# SQL QUERIES

### artifact_metadata Table


#### 1. List all artifacts from the 11th century belonging to Byzantine culture.

In [ ]:
cursor.execute(""" 
SELECT id, title, century, culture 
FROM artifact_metadata 
WHERE century LIKE '%11th century%' AND culture LIKE '%Byzantine%'; """) 
results = cursor.fetchall() 
for row in results: 
    print(row)

#### 2. What are the unique cultures represented in the artifacts?

In [ ]:
cursor.execute("""
    SELECT DISTINCT culture
    FROM artifact_metadata
    WHERE culture IS NOT NULL
    ORDER BY culture;
""")
results = cursor.fetchall()
print("Unique cultures in artifacts:")
for row in results:
    print(row[0])

#### 3. List all artifacts from the Archaic Period

In [ ]:
cursor.execute("""
SELECT id, title, period
FROM artifact_metadata
WHERE period = 'Archaic Period';
""")
results = cursor.fetchall()
print("Artifacts from Archaic Period:")
for row in results:
    print(row)


#### 4. List artifact titles ordered by accession year in descending order

In [ ]:
cursor.execute("""
    SELECT title, accessionyear
    FROM artifact_metadata
    WHERE accessionyear IS NOT NULL
    ORDER BY accessionyear DESC;
""")
results = cursor.fetchall()
print("Artifact titles ordered by accession year (newest first):")
for row in results:
    print(row)


#### 5. How many artifacts are there per department

In [ ]:
cursor.execute("""
SELECT department, COUNT(*) AS artifact_count
FROM artifact_metadata
GROUP BY department
ORDER BY artifact_count DESC;
""")
results = cursor.fetchall()
print("Number of artifacts per department:")
for department, count in results:
    print(f"{department}: {count}")

### artifact_media Table

#### 6. Which artifacts have more than 1 image

In [ ]:
cursor.execute("""
SELECT m.title, md.imagecount
FROM artifact_metadata m
JOIN artifact_media md ON m.id = md.objectid
WHERE md.imagecount > 1; """)
results = cursor.fetchall()
print("Artifacts with more than 1 image:")
for title, imagecount in results:
    print(f"Title: {title}, Images: {imagecount}")

#### 7. What is the average rank of all artifacts

In [ ]:
cursor.execute("""
SELECT AVG(rank_value) AS average_rank
FROM artifact_media
WHERE rank_value IS NOT NULL;
""")
result = cursor.fetchone()
print(f"Average rank of all artifacts: {result[0]}")

#### 8. Which artifacts have a higher colorcount than mediacount

In [ ]:
cursor.execute("""
SELECT m.title, am.colorcount, am.mediacount
FROM artifact_media am
JOIN artifact_metadata m ON am.objectid = m.id
WHERE am.colorcount > am.mediacount;
""")
results = cursor.fetchall()
print("Artifacts where color count is higher than media count:")
for title, colorcount, mediacount in results:
    print(f"Title: {title}, Color Count: {colorcount}, Media Count: {mediacount}")


#### 9. List all artifacts created between 1500 and 1600

In [ ]:
cursor.execute("""
    SELECT title, datebegin, dateend
    FROM artifact_metadata m
    JOIN artifact_media me ON m.id = me.objectid
    WHERE (me.datebegin BETWEEN 1500 AND 1600)
       OR (me.dateend BETWEEN 1500 AND 1600);
""")
results = cursor.fetchall()
print("Artifacts created between 1500 and 1600:")
for row in results:
    print(row)

#### 10. How many artifacts have no media files

In [ ]:
cursor.execute("""
    SELECT COUNT(*)
    FROM artifact_media
    WHERE mediacount = 0 OR mediacount IS NULL;
""")
no_media_count = cursor.fetchone()[0]
print(f"Artifacts with no media files: {no_media_count}")


### artifact_colors Table

#### 11. What are all the distinct hues used in the dataset

In [ ]:
cursor.execute("""
    SELECT DISTINCT hue
    FROM artifact_colors
    WHERE hue IS NOT NULL
    ORDER BY hue;
""")
results = cursor.fetchall()
print("Distinct hues:")
for row in results:
    print(row[0])


#### 12. What are the top 5 most used colors by frequency

In [ ]:
cursor.execute("""
SELECT color, COUNT(*) AS frequency
FROM artifact_colors
WHERE color IS NOT NULL
GROUP BY color
ORDER BY frequency DESC
LIMIT 5;
""")
results = cursor.fetchall()
print("Top 5 most used colors:")
for color, freq in results:
    print(f"{color}: {freq}")


#### 13. What is the average coverage percentage for each hue

In [ ]:
cursor.execute("""
    SELECT hue, AVG(percent) AS avg_percent
    FROM artifact_colors
    WHERE percent IS NOT NULL
    GROUP BY hue
    ORDER BY avg_percent DESC;
""")
results = cursor.fetchall()
print("Average coverage percentage per hue:")
for hue, avg in results:
    print(f"{hue}: {avg:.2f}%")


#### 14. List all colors used for a given artifact ID

In [ ]:
artifact_id = int(input("Enter Artifact ID: "))
cursor.execute("""
SELECT color, hue, percent, spectrum, css3
FROM artifact_colors
WHERE objectid = %s;
""", (artifact_id,))
results = cursor.fetchall()
print(f"Colors used for artifact ID {artifact_id}:")
for color, hue, percent, spectrum, css3 in results:
    print(f"{color} | Hue: {hue} | Percent: {percent} | Spectrum: {spectrum} | CSS3: {css3}")

Colors used for artifact ID 148258:
#e1e1fa | Hue: Violet | Percent: 0.32876190476190476 | Spectrum: #8c5fa8 | CSS3: #e6e6fa
#c8c8e1 | Hue: Blue | Percent: 0.30214285714285716 | Spectrum: #8761aa | CSS3: #d8bfd8
#fafafa | Hue: White | Percent: 0.2904761904761905 | Spectrum: #955ba5 | CSS3: #fffafa
#323232 | Hue: Grey | Percent: 0.04395238095238095 | Spectrum: #2eb45d | CSS3: #2f4f4f
#4b4b4b | Hue: Grey | Percent: 0.013095238095238096 | Spectrum: #3db657 | CSS3: #2f4f4f
#c8967d | Hue: Grey | Percent: 0.006333333333333333 | Spectrum: #e66c64 | CSS3: #bc8f8f
#646464 | Hue: Grey | Percent: 0.004047619047619047 | Spectrum: #7866ad | CSS3: #696969
#c87d64 | Hue: Orange | Percent: 0.0036666666666666666 | Spectrum: #e66c64 | CSS3: #cd5c5c
#e1af96 | Hue: Orange | Percent: 0.003 | Spectrum: #e9715f | CSS3: #deb887
#afafaf | Hue: Grey | Percent: 0.0013333333333333333 | Spectrum: #8c5fa8 | CSS3: #a9a9a9


#### 15. What is the total number of color entries in the dataset

In [ ]:
cursor.execute("""
SELECT COUNT(*) 
FROM artifact_colors;
""")
total_colors = cursor.fetchone()[0]
print(f"Total number of color entries in the dataset: {total_colors}")

### Join-Based Queries

#### 16. List artifact titles and hues for all artifacts belonging to the Byzantine culture.

In [ ]:
cursor.execute("""
SELECT m.title, c.hue
FROM artifact_metadata m
JOIN artifact_colors c ON m.id = c.objectid
WHERE m.culture = 'Byzantine';
""")
results = cursor.fetchall()
print("Artifacts from Byzantine culture with their hues:")
for row in results:
    print(f"Title: {row[0]}, Hue: {row[1]}")


#### 17. List each artifact title with its associated hues. 

In [ ]:
cursor.execute("""
SELECT md.title, GROUP_CONCAT(ac.hue SEPARATOR ', ') AS hues
FROM artifact_metadata md
JOIN artifact_colors ac ON md.id = ac.objectid
GROUP BY md.title;
""")
results = cursor.fetchall()
print("Artifact titles with their associated hues:")
for title, hues in results:
    print(f"{title}: {hues}")


#### 18. Get artifact titles, cultures, and media ranks where the period is not null.

In [ ]:
cursor.execute("""
SELECT md.title, md.culture, am.rank_value
FROM artifact_metadata md
JOIN artifact_media am ON md.id = am.objectid
WHERE md.period IS NOT NULL;
""")
results = cursor.fetchall()
for title, culture, rank in results:
    print(f"{title} | Culture: {culture} | Rank: {rank}")
    

#### 19. Find artifact titles ranked in the top 10 that include the color hue "Grey".

In [ ]:
cursor.execute("""
SELECT md.title, am.rank_value, ac.hue
FROM artifact_metadata md
JOIN artifact_media am ON md.id = am.objectid
JOIN artifact_colors ac ON md.id = ac.objectid
WHERE ac.hue = 'Grey'
ORDER BY am.rank_value ASC
LIMIT 10;
""")
results = cursor.fetchall()
print("Top 10 ranked artifacts with hue 'Grey':")
for title, rank, hue in results:
    print(f"{title} | Rank: {rank} | Hue: {hue}")


#### 20. How many artifacts exist per classification, and what is the average media count for each?

In [ ]:
cursor.execute("""
SELECT md.classification, 
       COUNT(*) AS artifact_count, 
       AVG(am.mediacount) AS avg_media_count
FROM artifact_metadata md
JOIN artifact_media am ON md.id = am.objectid
GROUP BY md.classification
ORDER BY artifact_count DESC;
""")
results = cursor.fetchall()
print("Artifacts per classification and average media count:")
for classification, count, avg_media in results:
    print(f"{classification}: Count = {count}, Average Media Count = {avg_media:.2f}")


### Extra SQL queries

#### 21. Find all colors for one artifact

In [ ]:
cursor.execute("""
    SELECT m.title, c.color, c.percent
    FROM artifact_metadata m
    JOIN artifact_colors c ON m.id = c.objectid
    ORDER BY c.percent DESC
    LIMIT 20;
""")
print("Top colors for artifacts:")
for row in cursor.fetchall():
    print(row)

#### 22. Average colors per classification

In [ ]:
cursor.execute("""
    SELECT m.classification, AVG(me.colorcount) AS avg_colors
    FROM artifact_metadata m
    JOIN artifact_media me ON m.id = me.objectid
    GROUP BY m.classification
    ORDER BY avg_colors DESC
    LIMIT 10;
""")
print("Average colors per classification:")
for row in cursor.fetchall():
    print(row)

#### 23. List all artifact titles that have "coin" in them

In [ ]:
cursor.execute("""
    SELECT title
    FROM artifact_metadata
    WHERE title LIKE '%coin%';
""")
results = cursor.fetchall()
print("Artifacts with 'coin' in the title:")
for row in results:
    print(row[0])

#### 24. Count how many artifacts belong to each accession year

In [ ]:
cursor.execute("""
    SELECT accessionyear, COUNT(*) AS count
    FROM artifact_metadata
    WHERE accessionyear IS NOT NULL
    GROUP BY accessionyear
    ORDER BY accessionyear;
""")
results = cursor.fetchall()
print("Number of artifacts by accession year:")
for row in results:
    print(row)

#### 25. List all unique mediums used

In [ ]:
cursor.execute("""
    SELECT DISTINCT medium
    FROM artifact_metadata
    WHERE medium IS NOT NULL
    ORDER BY medium;
""")
results = cursor.fetchall()
print("Unique mediums:")
for row in results:
    print(row[0])